In [1]:
from geopy.geocoders import GoogleV3
import numpy as np
import geocoder
import json
import pandas as pd
from geojson import Feature, FeatureCollection, Point
import tabula
import re

In [2]:
# Gotta figure out how to get rid of the java error.
# SO: https://stackoverflow.com/questions/54817211/java-command-is-not-found-from-this-python-process-please-ensure-java-is-inst

dsf = tabula.read_pdf('condo.pdf', pages='all',lattice=True)

In [3]:
df = dsf[0]
df.columns = df.iloc[0]
df = df.drop(df.index[0])
### This next line gets ride of the Nan ###
df = df.iloc[: , 1:]
df = df.replace(np.nan, 'Not Available', regex=True)
df = df.rename(columns={'Building / Address / City': 'building_address_city','Days on\rMarket':'days_on_market',\
                        'Price /\rSq. Ft.':'price_per_sqft'})
df['geo_address'] = df['building_address_city']
df['geo_address'] = df['geo_address'].map(lambda x: re.sub(r'\r', ' ', x))
df = df.replace('\n',' ',regex=True)
df2 = df['geo_address'].str.extract(r'(?P<building_name>\D*)(?P<address_only>\d.*)', expand=True)

Review this later (it's about how to properly merge DFs): https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

In [4]:
result = pd.merge(df,df2,left_index=True, right_index=True)
df = result

In [5]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [6]:
df['loc'] = df['address_only'].apply(geolocator.geocode, timeout=10)

In [7]:
df['loc']

1     (FAENA HOUSE, 3315 Collins Ave #6a, Miami Beac...
2     (1643 Brickell Ave # 4902, Miami, FL 33129, US...
3     (10201 Collins Ave Unit 2103, Bal Harbour, FL ...
4     (10203 Collins Ave Unit 1602, Bal Harbour, FL ...
5     (50 South Pointe Dr APT 3104, Miami Beach, FL ...
6     (360 Ocean Dr #501s, Key Biscayne, FL 33149, U...
7     (1000 Biscayne Blvd #3201, Miami, FL 33132, US...
8     (3471 Main Hwy APT 727, Miami, FL 33133, USA, ...
9     (5132 Fisher Island Dr #5132, Miami Beach, FL ...
10    (18555 Collins Ave, Sunny Isles Beach, FL 3316...
Name: loc, dtype: object

In [8]:
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)

Columns must be same length as key explained
- https://cumsum.wordpress.com/2021/06/01/pandas-valueerror-columns-must-be-same-length-as-key/

Creating `df2` based on `df` row indicies.

https://datascience.stackexchange.com/questions/77033/creating-a-new-dataframe-with-specific-row-numbers-from-another

What's tripping this error is when the lat/lon/alititude generator is fed a Nan value to parse - it's effectively trying to split a `null` value into three separate parts - which is impossible.

The code below effectively caputres the anatomy of this error. It creates a new `df` based on a list of indicies from an existing one. Row `56` has no `point` since the `geocoder` couldn't find coordinates for it. The other three do. Splitting the `point` column fails only when row `56` - with `null` values - is attempted.
```
df2 = df.iloc[[56,57,58,59],:]
df2[['lat','lon','altitude']] = pd.DataFrame(df2['point'].to_list(),index=df2.index)
```

In [9]:
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

df

,building_address_city,List Date,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only,loc,point,lat,lon,altitude
1,Faena House\r3315 Collins Ave 6a\rMiami Beach,1/31/2022,5/4/2022,63,"$15,500,000.00",6246,"$3,276.96",Toni Schrager,Brown Harris Stevens,Bryan Sereny PA,Douglas Elliman,Faena House 3315 Collins Ave 6a Miami Beach,Faena House,3315 Collins Ave 6a Miami Beach,"(FAENA HOUSE, 3315 Collins Ave #6a, Miami Beac...","(25.8078488, -80.1232302, 0.0)",25.807849,-80.123230,0.0
2,Santa Maria\r1643 Brickell Ave PH 4902\rMiami,5/10/2021,5/5/2022,309,"$12,000,000.00",10000,"$1,200.00",Audrey Ross,"Compass Florida, LLC",Liz Hogan,"Compass Florida, LLC",Santa Maria 1643 Brickell Ave PH 4902 Miami,Santa Maria,1643 Brickell Ave PH 4902 Miami,"(1643 Brickell Ave # 4902, Miami, FL 33129, US...","(25.755254, -80.194744, 0.0)",25.755254,-80.194744,0.0
3,Oceana Bal Harbour Condo\r10201 Collins Ave 21...,9/22/2021,5/5/2022,201,"$8,900,000.00",3264,"$2,726.72",Jonathan Mann,Coldwell Banker Realty,Omer Horev,Charles Rutenberg Realty,Oceana Bal Harbour Condo 10201 Collins Ave 210...,Oceana Bal Harbour Condo,10201 Collins Ave 2103 Bal Harbour,"(10201 Collins Ave Unit 2103, Bal Harbour, FL ...","(25.8951782, -80.1231506, 0.0)",25.895178,-80.123151,0.0
4,Oceana Bal Harbour Condo\r10203 Collins Ave 16...,5/1/2022,5/2/2022,1,"$8,900,000.00",3450,"$2,579.71",Comp Sale,Comp Sale,Lourdes Alatriste,Douglas Elliman,Oceana Bal Harbour Condo 10203 Collins Ave 160...,Oceana Bal Harbour Condo,10203 Collins Ave 1602 Bal Harbour,"(10203 Collins Ave Unit 1602, Bal Harbour, FL ...","(25.8951782, -80.1231506, 0.0)",25.895178,-80.123151,0.0
5,Continuum on South Beach\r50 S Pointe Dr 3104\...,4/6/2022,5/2/2022,2,"$8,800,000.00",2173,"$4,049.70",Jorge Martinez,One Sotheby's Intl Realty,Devin Kay,Douglas Elliman,Continuum on South Beach 50 S Pointe Dr 3104 M...,Continuum on South Beach,50 S Pointe Dr 3104 Miami Beach,"(50 South Pointe Dr APT 3104, Miami Beach, FL ...","(25.7682956, -80.1316247, 0.0)",25.768296,-80.131625,0.0
6,Oceana Key Biscayne\r360 Ocean Dr 501S\rKey Bi...,8/11/2021,5/5/2022,253,"$8,800,000.00",3791,"$2,321.29",Brigitte Nachtigall,Great Properties Intl,David Deleon,"Compass Florida, LLC",Oceana Key Biscayne 360 Ocean Dr 501S Key Bisc...,Oceana Key Biscayne,360 Ocean Dr 501S Key Biscayne,"(360 Ocean Dr #501s, Key Biscayne, FL 33149, U...","(25.6942877, -80.1578218, 0.0)",25.694288,-80.157822,0.0
7,One Thousand Museum\r1000 E Biscayne Blvd 3201...,4/26/2022,5/4/2022,1,"$7,300,000.00",4600,"$1,586.96",Patrice Hallot,London Foster Realty,Anna Sherrill,One Sotheby's Intl Realty,One Thousand Museum 1000 E Biscayne Blvd 3201 ...,One Thousand Museum,1000 E Biscayne Blvd 3201 Miami,"(1000 Biscayne Blvd #3201, Miami, FL 33132, US...","(25.784171, -80.1900783, 0.0)",25.784171,-80.190078,0.0
8,Cloisters on the Bay Condo\r3471 Main Hwy 727\...,2/28/2022,5/2/2022,7,"$7,000,000.00",5739,"$1,219.72",Ilana Levitt,"Compass Florida, LLC",Ilana Levitt,"Compass Florida, LLC",Cloisters on the Bay Condo 3471 Main Hwy 727 M...,Cloisters on the Bay Condo,3471 Main Hwy 727 Miami,"(3471 Main Hwy APT 727, Miami, FL 33133, USA, ...","(25.7260243, -80.2424968, 0.0)",25.726024,-80.242497,0.0
9,Bayview at Fisher Island\r5132 Fisher Island D...,11/16/2021,5/2/2022,121,"$6,500,000.00",3580,"$1,815.64",Riley Smith,BHHS EWM Realty,Riley Smith,BHHS EWM Realty,Bayview at Fisher Island 5132 Fisher Island Dr...,Bayview at Fisher Island,5132 Fisher Island Dr 5132 Miami Beach,"(5132 Fisher Island Dr #5132, Miami Beach, FL ...","(25.7620721, -80.1477688, 0.0)",25.762072,-80.147769,0.0
10,P0orsche Design Tower\r18555 Collins Ave 2501\...,1/11/2022,5/2/2022,51,"$6,050,000.00",5913,"$1,340.57",Enrique Tettamanti,Dezer Platinum Realty,Melissa Barragan,Dezer Platinum Realty,P0orsche Design Tower 18555 Collins Ave 2501 S...,P,0orsche Design Tower 18555 Collins Ave 2501 Su...,"(18555 Collins Ave, Sunny Is

Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

In [10]:
df.columns

Index(['building_address_city', 'List Date', 'Close Date', 'days_on_market',
       'Sale Price', 'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker',
       'Buyer Agent', 'Buyer Broker', 'geo_address', 'building_name',
       'address_only', 'loc', 'point', 'lat', 'lon', 'altitude'],
      dtype='object')

In [15]:
df.at[10,'building_name']=('Porsche Design Tower')
df.at[10,'address_only']=('18555 Collins Ave 2501')
# df.at[8,'building_name']=('18975 Collins Ave 3403')
# df.at[8,'address_only']=('18975 Collins Ave 3403')
# df.at[6,'building_name']=('The Ritz-Carlton Residences, Miami Beach')

In [12]:
df.columns

Index(['building_address_city', 'List Date', 'Close Date', 'days_on_market',
       'Sale Price', 'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker',
       'Buyer Agent', 'Buyer Broker', 'geo_address', 'building_name',
       'address_only', 'loc', 'point', 'lat', 'lon', 'altitude'],
      dtype='object')

In [16]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=11)
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format('May 1st - May 7th')

marker_cluster = MarkerCluster().add_to(m)

for i,r in df.iterrows():
    location = (r["lat"], r["lon"])
    info = (r["Sale Price"], r["days_on_market"], r["Agent"], r['Buyer Agent'], r['price_per_sqft'], r['address_only'], r['building_name'])
    info = list(info)
    new_line = '<br>'
    bold_start = '<strong>'
    bold_end = '</strong>'
    text = f'Building Name: {bold_start}{info[-1]}{bold_end}{new_line} \
    Sale Price: {bold_start}{info[0]}{bold_end}{new_line}Days on Market: \
    {bold_start}{info[1]}{bold_end}{new_line}Listing Agent: {bold_start}{info[2]} \
    {bold_end}{new_line}Buying Agent: {bold_start}{info[3]}{bold_end}{new_line}Price sq ft: \
    {bold_start}{info[4]}{bold_end}{new_line}Address: {bold_start}{info[5]}{bold_end}'
    folium.Marker(location=location,
                      tooltip=text)\
    .add_to(marker_cluster)

m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))

m

In [17]:
m.save('index.html')